In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

# 1. Collect course codes from program page

In [2]:
url = "https://laurentian.ca/program/mechanical-engineering/details"

In [3]:
uClient = ureq(url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")

In [4]:
start = page_soup.find("div", {"id": "options"}).find("p", text="B.Eng. in Mechanical Engineering - Mechatronics Option")
start

<p><strong>B.Eng. in Mechanical Engineering - Mechatronics Option</strong></p>

In [6]:
import bs4
course_titles = []
counter = 0

while start != "Mechanical Engineering Technical Electives – List A":
    if type(start) is bs4.element.NavigableString:
        course_title = " ".join(re.split("\s+", start.strip()))
        if bool(re.match("[A-Z]{4} [0-9]{4}E", course_title)):
            course_titles.append(course_title)
            print(course_titles[-1])
            counter += 1
    start = start.next_element

CHMI 1006E General Chemistry I
ENGR 1007E Engineering Graphics and Design
ENGR 1056E Applied Mechanics I
MATH 1036E Calculus I
PHYS 1006E Introductory Physics I
ENGR 1016E Engineering Communications
ENGR 1017E Computer Aided Design
ENGR 1057E Applied Mechanics II
MATH 1037E Calculus II
MATH 1057E Linear Algebra I
PHYS 1007E Introductory Physics II
COSC 1046E Computer Science I
ENGR 2026E Engineering Materials
ENGR 2036E Engineering Thermodynamics I
ENGR 2076E Mechanics and Strength of Materials I
MATH 2066E Introduction to Differential Equations
COSC 1047E Computer Science II
ENGR 2097E Fluid Mechanics I
ENGR 2506E Dynamics
ENGR 2537E Engineering Thermodynamics II
ENGR 2587E Mechanics and Strength of Materials II
STAT 2246E Statistics for Scientists
COSC 2006E Data Structures
ENGR 3426E Engineering Economics
ENGR 3526E Fluid Mechanics II
ENGR 3536E Materials and Manufacturing
ENGR 3546E System Modeling and Simulation
ENGR 3556E Electromechanical Actuators and Power Electronics
ENGR 350

In [7]:
len(course_titles)

44

# 2. Scrape course descriptions from courses website

In [8]:
url = "https://laurentian.ca/courses"

In [9]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [10]:
driver.get(url)

In [11]:
page_soup = soup(driver.page_source, 'lxml')

In [12]:
from selenium.common.exceptions import NoSuchElementException

course_names = []
course_descs = []
course_codes = []
counter = 0

for course_title in course_titles:
    
    #type in search box
    driver.find_elements_by_tag_name("input")[1].send_keys(course_title)
    time.sleep(1)
    
    #click on course link
    try:
        driver.find_element_by_link_text("More Info").click()
        time.sleep(3)
    except NoSuchElementException:
        driver.find_elements_by_tag_name("input")[1].clear()
        time.sleep(1)
        driver.find_elements_by_tag_name("input")[1].send_keys(" ".join(course_title.split()[:2]))
        time.sleep(1)
        
        try:
            driver.find_element_by_link_text("More Info").click()
            time.sleep(3)
        except NoSuchElementException:
            print("Cannot find ", course_title)
            driver.find_elements_by_tag_name("input")[1].clear()
            time.sleep(1)
            continue
    
    #switch to new tab
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(2)
    
    #get html and scrape
    page_soup = soup(driver.page_source, 'lxml')
    container = page_soup.find("div", {"class": "region region-content"}).find("div", {"class": "container"})
    
    course_name = container.h1.text.strip()
    course_desc = container.find("p").text.strip().split("PREREQ:")[0]
    course_code = " ".join(course_title.split()[:2])
    
    course_names.append(course_name)
    course_descs.append(course_desc)
    course_codes.append(course_code)
    
    counter += 1
    print("scraped ", course_codes[-1])
    
    #close tab
    driver.close()
    time.sleep(1)
    
    #switch to original tab
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(1)
    
    #clear search box
    driver.find_elements_by_tag_name("input")[1].clear()
    time.sleep(1)
    
print(counter)

scraped  CHMI 1006E
scraped  ENGR 1007E
scraped  ENGR 1056E
scraped  MATH 1036E
scraped  PHYS 1006E
scraped  ENGR 1016E
scraped  ENGR 1017E
scraped  ENGR 1057E
scraped  MATH 1037E
scraped  MATH 1057E
scraped  PHYS 1007E
scraped  COSC 1046E
scraped  ENGR 2026E
scraped  ENGR 2036E
scraped  ENGR 2076E
scraped  MATH 2066E
scraped  COSC 1047E
scraped  ENGR 2097E
scraped  ENGR 2506E
scraped  ENGR 2537E
scraped  ENGR 2587E
scraped  STAT 2246E
scraped  COSC 2006E
scraped  ENGR 3426E
scraped  ENGR 3526E
scraped  ENGR 3536E
scraped  ENGR 3546E
scraped  ENGR 3556E
scraped  ENGR 3507E
scraped  ENGR 3516E
scraped  ENGR 3517E
scraped  ENGR 3547E
scraped  ENGR 3577E
scraped  MATH 3416E
scraped  ENGR 3446E
scraped  ENGR 4126E
scraped  ENGR 4526E
scraped  ENGR 4546E
scraped  ENGR 4576E
scraped  ENGR 4595E
scraped  COSC 3807E
scraped  ENGR 4117E
scraped  ENGR 4547E
scraped  ENGR 4595E
44


# 3. Inspect and write to CSV

In [13]:
course_codes

['CHMI 1006E',
 'ENGR 1007E',
 'ENGR 1056E',
 'MATH 1036E',
 'PHYS 1006E',
 'ENGR 1016E',
 'ENGR 1017E',
 'ENGR 1057E',
 'MATH 1037E',
 'MATH 1057E',
 'PHYS 1007E',
 'COSC 1046E',
 'ENGR 2026E',
 'ENGR 2036E',
 'ENGR 2076E',
 'MATH 2066E',
 'COSC 1047E',
 'ENGR 2097E',
 'ENGR 2506E',
 'ENGR 2537E',
 'ENGR 2587E',
 'STAT 2246E',
 'COSC 2006E',
 'ENGR 3426E',
 'ENGR 3526E',
 'ENGR 3536E',
 'ENGR 3546E',
 'ENGR 3556E',
 'ENGR 3507E',
 'ENGR 3516E',
 'ENGR 3517E',
 'ENGR 3547E',
 'ENGR 3577E',
 'MATH 3416E',
 'ENGR 3446E',
 'ENGR 4126E',
 'ENGR 4526E',
 'ENGR 4546E',
 'ENGR 4576E',
 'ENGR 4595E',
 'COSC 3807E',
 'ENGR 4117E',
 'ENGR 4547E',
 'ENGR 4595E']

In [14]:
course_names

['General Chemistry I',
 'Engineering Graphics and Design',
 'Applied Mechanics I',
 'Calculus I',
 'Introductory Physics I',
 'Engineering Communications',
 'Computer Aided Design',
 'Applied Mechanics II',
 'Calculus II',
 'Linear Algebra I',
 'Introductory Physics II',
 'Computer Science 1',
 'Engineering Materials',
 'Engineering Thermodynamics I',
 'Mechanics and Strength of Materials I',
 'Introduction to Differential Equations',
 'Computer Science II',
 'Fluid Mechanics I',
 'Dynamics',
 'Engineering Thermodynamics II',
 'Mechanics and Strength of Materials II',
 'Statistics for Scientists',
 'Data Structures',
 'Engineering Economics',
 'Fluid Mechanics II',
 'Materials and Manufacturing',
 'System Modeling and Simulation',
 'Electromechanical Actuators',
 'Finite Element Methods',
 'Vibrations and Dynamic Systems',
 'Machine Design',
 'Control Systems',
 'Microprocessor Systems and Interfacing',
 'Numerical Methods I',
 'Heat Transfer',
 'Engineering Seminar',
 'Sensors and In

In [15]:
course_descs

['This course provides an introduction to the fundamental principles of chemistry. Topics include atomic and molecular structure, nuclear chemistry, stoichiometry, the periodic table and periodicity, and the properties of solutions. ',
 'This course develops abilities for three-dimensional visualization and introduces the standards employed in graphical communication and engineering design. Drawings are prepared by freehand sketching and commercial computer-aided drafting and design (CADD) programs. Drafting procedures and standards for major engineering disciplines are described and explained, including orthographic, sectional and pictorial views, dimensional, descriptive geometry, tolerances, working drawings and flowcharts. A group design project is conducted for student production of a 3-dimensional computer model of the design, in addition to a set of working drawings. (lec 2, lab 2) cr 3.',
 'Topics include statics of particles, rigid bodies, equilibrium, centroid and centre of g

In [18]:
len(list(dict.fromkeys(course_codes)))

43

eliminate a duplicate engineering design project course that's repeated over two terms.

In [19]:
course_codes = course_codes[:-1]
course_names = course_names[:-1]
course_descs = course_descs[:-1]

In [20]:
course_descs = [desc.strip() for desc in course_descs]

In [21]:
import pandas as pd

df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,CHMI 1006E,General Chemistry I,This course provides an introduction to the fu...
1,ENGR 1007E,Engineering Graphics and Design,This course develops abilities for three-dimen...
2,ENGR 1056E,Applied Mechanics I,"Topics include statics of particles, rigid bod..."
3,MATH 1036E,Calculus I,This is a problem-solving course which include...
4,PHYS 1006E,Introductory Physics I,This is the first of two courses in introducto...
5,ENGR 1016E,Engineering Communications,This course offers an introduction to oral and...
6,ENGR 1017E,Computer Aided Design,Computer-aided design (CAD) is a tool that is ...
7,ENGR 1057E,Applied Mechanics II,Topics include kinematics and kinetics of part...
8,MATH 1037E,Calculus II,This course begins with further techniques of ...
9,MATH 1057E,Linear Algebra I,"Topics include simultaneous linear equations, ..."


In [22]:
df.to_csv('Laurentian_MechEng_Mechatronics_Option_Core_(AllYears)_Courses.csv', index = False)

In [23]:
driver.quit()